In [1]:
import numpy as np
import gzip
import pickle
from keras.models import load_model

Using TensorFlow backend.


In [2]:
f = gzip.GzipFile('kfold/all_dataset_window19Middle.npy.gz', "r")
dataset = np.load(f)
print(dataset.shape)

(1133502, 19, 50)


In [2]:
path = "699-Guilermo-Dropout-0.5.hdf5"
m = load_model( path )

In [8]:
filename = "munge-dataset1"
fp = np.memmap(filename, dtype='float64', mode='w+', shape=(1133502 * 2, 19, 50))

In [9]:
fp[0:1133502] = dataset
print(fp.shape)

(2267004, 19, 50)


In [10]:
fp.flush()
del fp

In [13]:
fp = np.memmap(filename, dtype='float64', mode='r', shape=(1133502 * 2, 19, 50))

In [14]:
windowSize = 19 #19
classSize = 8  # 2 or 3

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv1D, AveragePooling1D,MaxPooling1D, TimeDistributed, LeakyReLU, BatchNormalization, Flatten, Input, concatenate
from keras import optimizers, callbacks
from keras.regularizers import l2
from keras.optimizers import Adagrad, Adam
import tensorflow as tf

LR = 0.0009 # maybe after some (10-15) epochs reduce it to 0.0008-0.0007
drop_out = 0.5
batch_dim = 64
nn_epochs = 20
w_reg = l2(0) # 0.0003)
number_filters = 16

loss = 'categorical_crossentropy'



input_shape = (windowSize, 21)

conv1_input = Input(shape=(windowSize, 21), name='InputWindow')

conv_1 = Conv1D( 64 , 19,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network1-filter1')(conv1_input)
conv_1 = BatchNormalization(name='BN1')(conv_1)
conv_1 = Dropout(drop_out)(conv_1)
conv_2 = Conv1D( 64 , 11,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network1-filter2')(conv1_input)
conv_2 = BatchNormalization(name='BN2')(conv_2)
conv_2 = Dropout(drop_out)(conv_2)
conv_3 = Conv1D( 64 , 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network1-filter3')(conv1_input)
conv_3 = BatchNormalization(name='BN3')(conv_3)
conv_3 = Dropout(drop_out)(conv_3)

merge_1 = concatenate([conv_1, conv_2, conv_3], name='Network1')
input_for_second = concatenate([conv1_input, merge_1], name='Network1-and-input')



conv_4 = Conv1D( 64 , 19,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network2-filter1')(input_for_second)
conv_4 = BatchNormalization(name='BN4')(conv_4)
conv_4 = Dropout(drop_out)(conv_4)
conv_5 = Conv1D( 64 , 11,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network2-filter2')(input_for_second)
conv_5 = BatchNormalization(name='BN5')(conv_5)
conv_5 = Dropout(drop_out)(conv_5)
conv_6 = Conv1D( 64 , 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network2-filter3')(input_for_second)
conv_6 = BatchNormalization(name='BN6')(conv_6)
conv_6 = Dropout(drop_out)(conv_6)

merge_2 = concatenate([conv_4, conv_5, conv_6], name='Network2')
input_for_third = concatenate([conv1_input, merge_1, merge_2],name='Network1-Network2-and-input')



conv_7 = Conv1D( 64 , 19,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network3-filter1')(input_for_third)
conv_7 = BatchNormalization(name='BN7')(conv_7)
conv_7 = Dropout(drop_out)(conv_7)
conv_8 = Conv1D( 64 , 11,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network3-filter2')(input_for_third)
conv_8 = BatchNormalization(name='BN8')(conv_8)
conv_8 = Dropout(drop_out)(conv_8)
conv_9 = Conv1D( 64 , 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network3-filter3')(input_for_third)
conv_9 = BatchNormalization(name='BN9')(conv_9)
conv_9 = Dropout(drop_out)(conv_9)

merge_3 = concatenate([conv_7, conv_8, conv_9],name='Network3')
input_for_4 = concatenate([conv1_input, merge_1, merge_2, merge_3],name='Network123-and-input')



conv_10 = Conv1D( 64 , 19,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network4-filter1')(input_for_4)
conv_10 = BatchNormalization(name='BN10')(conv_10)
conv_10 = Dropout(drop_out)(conv_10)
conv_11 = Conv1D( 64 , 11,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network4-filter2')(input_for_4)
conv_11 = BatchNormalization(name='BN11')(conv_11)
conv_11 = Dropout(drop_out)(conv_11)
conv_12 = Conv1D( 64 , 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network4-filter3')(input_for_4)
conv_12 = BatchNormalization(name='BN12')(conv_12)
conv_12 = Dropout(drop_out)(conv_12)

merge_4 = concatenate([conv_10, conv_11, conv_12],name='Network4')
input_for_5 = concatenate([conv1_input, merge_1, merge_2, merge_3, merge_4],name='Network1234-and-input')



conv_13 = Conv1D( 64 , 19,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network5-filter1')(input_for_5)
conv_13 = BatchNormalization(name='BN13')(conv_13)
conv_13 = Dropout(drop_out)(conv_13)
conv_14 = Conv1D( 64 , 11,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network5-filter2')(input_for_5)
conv_14 = BatchNormalization(name='BN14')(conv_14)
conv_14 = Dropout(drop_out)(conv_14)
conv_15 = Conv1D( 64 , 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg, name='Network5-filter3')(input_for_5)
conv_15 = BatchNormalization(name='BN15')(conv_15)
conv_15 = Dropout(drop_out)(conv_15)

merge_5 = concatenate([conv_13, conv_14, conv_15],name='Network5')



merge_final = concatenate([merge_1, merge_2, merge_3,merge_4, merge_5], name='Final')



flatten  = Flatten()(merge_final)
first_dense = Dense(128, activation='relu', use_bias=True,  kernel_regularizer=w_reg, name='last')(flatten)
first_dense = BatchNormalization(name='BN16')(first_dense)

second_dense = Dense(32, activation='relu', use_bias=True,  kernel_regularizer=w_reg, name='last')(flatten)
second_dense = BatchNormalization(name='BN16')(second_dense)

final_model_output = Dense(classSize, activation = 'softmax', name='softmax')(first_dense)

m = Model(inputs=conv1_input, outputs=final_model_output)

opt = Adam(lr=LR)
m.compile(optimizer=opt, loss=loss,metrics=['accuracy', 'mae'])

print("\nHyper Parameters\n")
print("Learning Rate: " + str(LR))
print("Drop out: " + str(drop_out))
print("Batch dim: " + str(batch_dim))
print("Number of epochs: " + str(nn_epochs))
print("Regularizers: " + str(w_reg.l2))
print("\nLoss: " + loss + "\n")
m.summary()

#import os
#os.environ["PATH"] += os.pathsep + 'C:/Users/Ieremie/Anaconda3/pkgs/graphviz-2.38-hfd603c8_2/Library/bin'
#from keras.utils import plot_model
#plot_model(m)#, to_file='model.png')


Hyper Parameters

Learning Rate: 0.0009
Drop out: 0.5
Batch dim: 64
Number of epochs: 20
Regularizers: 0.0

Loss: categorical_crossentropy

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
InputWindow (InputLayer)        (None, 19, 21)       0                                            
__________________________________________________________________________________________________
Network1-filter1 (Conv1D)       (None, 19, 64)       25600       InputWindow[0][0]                
__________________________________________________________________________________________________
Network1-filter2 (Conv1D)       (None, 19, 64)       14848       InputWindow[0][0]                
__________________________________________________________________________________________________
Network1-filter3 (Conv1D)       (None, 19, 64)    

In [16]:
from keras.callbacks import ModelCheckpoint

filepath="best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)
callbacks_list = [checkpoint]

In [17]:
from time import time
from timeit import default_timer as timer

start_time = timer()
history = m.fit(dataset[:1133502,:,21+classSize:], dataset[:1133502,0,21 : (21+classSize)], epochs=nn_epochs, batch_size=batch_dim,
                shuffle=True) #,  callbacks=callbacks_list)

end_time = timer()
print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")

Epoch 1/20
  38848/1133502 [>.............................] - ETA: 31:27 - loss: 1.1892 - accuracy: 0.5975 - mae: 0.1264

KeyboardInterrupt: 